In [ ]:
# Initial imports.
import pandas as pd
import numpy as np
from numpy import random
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Import RDS password
from config import rds_pwd

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graph

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-13 03:35:29--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.5’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-05-13 03:35:30 (11.6 MB/s) - ‘postgresql-42.2.16.jar.5’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://wine-final-project.czqkltznl3rl.us-east-2.rds.amazonaws.com/winedb"
#config = {"user":"postgres",
          #"password": "<password>",
          #"driver":"org.postgresql.Driver"}
config = {"user":"wineuser",
          "password": rds_pwd,
          "driver":"org.postgresql.Driver"}

In [ ]:
select_sql = "(SELECT points, variety, avg_temp, avg_airp, avg_humid, avg_precip FROM machine"
wine_df = spark.read.jdbc(url=jdbc_url, table='weather_mapscaled', properties=config)
#county_df = spark.read.jdbc(url=jdbc_url, table='counties', properties=config)
wine_df.show()

+--------------------+---+----+--------------------+--------------------+--------------------+--------------------+------------+------+------------------+--------------------+
|  winery_search_term|lat| lng|            avg_temp|            avg_airp|           avg_humid|          avg_precip|     country|points|           variety|              winery|
+--------------------+---+----+--------------------+--------------------+--------------------+--------------------+------------+------+------------------+--------------------+
|   LaZarre winery US| 36|-121|279.6850000000000...|1018.230000000000...|73.42000000000000...|0.025000000000000000|          US|    88|        Chardonnay|             LaZarre|
|Negretti winery I...| 45|   8|275.6025000000000...|1016.337500000000...|79.04000000000000...|0.070000000000000000|       Italy|    88|        Chardonnay|            Negretti|
|Pico Maccario win...| 45|   8|280.6950000000000...|1017.880000000000...|70.50250000000000...|0.147500000000000000|     

In [ ]:
allwines = wine_df.select('variety','lat','lng','avg_temp','avg_airp','avg_humid','avg_precip', (wine_df.points >= 90).alias('success'))
allwines.show()

+------------------+---+----+--------------------+--------------------+--------------------+--------------------+-------+
|           variety|lat| lng|            avg_temp|            avg_airp|           avg_humid|          avg_precip|success|
+------------------+---+----+--------------------+--------------------+--------------------+--------------------+-------+
|        Chardonnay| 36|-121|279.6850000000000...|1018.230000000000...|73.42000000000000...|0.025000000000000000|  false|
|        Chardonnay| 45|   8|275.6025000000000...|1016.337500000000...|79.04000000000000...|0.070000000000000000|  false|
|        Chardonnay| 45|   8|280.6950000000000...|1017.880000000000...|70.50250000000000...|0.147500000000000000|  false|
|        Chardonnay| 43|  12|278.1475000000000...|1014.445000000000...|81.86750000000000...|0.182500000000000000|  false|
|        Chardonnay| 43|  12|278.1475000000000...|1014.445000000000...|81.86750000000000...|0.182500000000000000|  false|
|        Chardonnay| 43|

In [ ]:
mer_df = allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','success').filter(allwines.variety == 'Merlot').collect()
#mer_df = wine_df.select(variety='Merlot').collect()
merlot = pd.DataFrame(mer_df, columns=['lat','lng','avg_temp','avg_airp','avg_humid','avg_precid','success'], dtype=float)
merlot

,lat,lng,avg_temp,avg_airp,avg_humid,avg_precid,success
0,36.0,-87.0,277.1825,1020.1425,77.9250,0.1875,0.0
1,36.0,-87.0,277.1825,1020.1425,77.9250,0.1875,0.0
2,42.0,-123.0,276.0250,1019.8875,85.4525,0.0575,0.0
3,44.0,-123.0,276.0250,1019.8875,85.4525,0.0575,0.0
4,36.0,-84.0,277.3750,1020.0125,79.1975,0.2350,0.0
...,...,...,...,...,...,...,...
2969,41.0,-72.0,272.7475,1016.8375,70.1425,0.1400,0.0
2970,41.0,-72.0,272.7475,1016.8375,70.1425,0.1400,0.0
2971,41.0,-72.0,272.7475,1016.8375,70.1425,0.1400,0.0
2972,35.0,-84.0,280.4425,1019.8050,73.0225,0.2600,0.0


In [ ]:
merlot.dtypes

lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precid    float64
success       float64
dtype: object

In [ ]:
#allwine = pd.DataFrame(allwines)
#allwine['success'] = (allwine['points'] >= 90).astype(int)

In [ ]:
#counties = '/content/uscounty_weather.csv'
#wineweather = '/content/weather_mapscaled.csv'
#county_df = pd.read_csv(counties, index_col=0)
#wine_df = pd.read_csv(wineweather)

In [ ]:
#county_df.columns

In [ ]:
#zipdrops = ['zip', 'city', 'state_id', 'state_name', 'county_name']

In [ ]:
#zipdata = county_df.dropna()
#zipdata.columns

In [ ]:
#wine_df['success'] = (wine_df['points'] >= 90).astype(int)
#wine_df['success'].value_counts()

In [ ]:
#wine_df.columns

In [ ]:
winedropcols = ['success']

In [ ]:
#learndropcols = ['latitude', 'longitude', 'Average Temperature (Kelvin)','Average Air Pressure (hPa)', 'Average Humidity (%)','Average Daily Precipitation (mm)', 'success']

In [ ]:
#def splitwines(winedata, point_thresh, winetype):
  
  #winedata['success'] = (winedata['points'] >= point_thresh).astype(int)
  #varieties = list(winedata['variety'].unique())
  #vardfs = {}
  #for variety in varieties:
  #  vardfs[variety] = winedata[winedata['variety'] == variety].drop(columns=winedropcols)
  #return vardfs[conv].columns


In [ ]:
#splitwines(wine_df, 90)

In [ ]:
#def learnwine (varietydict, variety, zipdata):
 # usewine = varietydict[variety]
  #y = usewine['success']
#  X = usewine.drop(columns='success')
 # X_train, X_test, y_train, y_test = train_test_split(X,y)
#  rf = RandomForestClassifier(n_estimators=200)
#  rf.fit(X,y)
#  testpreds = rf.predict(X_test)
#  acc_score = accuracy_score(y_test, testpreds)
#  counties = zipdata.drop(columns=dropcounties)
#  preds = rf.predict(counties)
#  return acc_score, preds

In [ ]:
#def biglearn (winedata, cutoff, winetype, zipdata):
#  countypreds = zipdata.copy()
#  targvar = str(winetype)
#  predcounties = zipdata['zip']
#  winedata['success'] = (winedata['points'] >= cutoff).astype(int)
#  varieties = list(winedata['variety'].unique())
#  vardfs = {}
#  for variety in varieties:
#    vardfs[variety] = winedata[winedata['variety'] == variety].drop(columns=winedropcols)
#  usewine = vardfs[targvar]
 # y = usewine['success']
 # X = usewine.drop(columns='success')
#  Xcounts = zipdata.drop(columns=['zip', 'city', 'state_id', 'state_name', 'county_name'])
#  X_train, X_test, y_train, y_test = train_test_split(X,y)
#  rf = RandomForestClassifier(n_estimators=200)
#  rf.fit(X,y)
#  testpreds = rf.predict(X_test)
#  acc_score = accuracy_score(y_test, testpreds)
#  cpreds = rf.predict(Xcounts)
#  countypreds[targvar] = cpreds
#    
#  return print(acc_score, countypreds[targvar].value_counts())

In [ ]:
#biglearn(wine_df, 89, 'Merlot', zipdata)

In [ ]:
#teststr('Merlot')

In [ ]:
county_weath = spark.read.jdbc(url=jdbc_url, table='uscounty', properties=config)
county_weath.show()

+----+---+---+------------+--------+-------------+------------+--------------------+--------------------+--------------------+--------------------+
| zip|lat|lng|        city|state_id|   state_name| county_name|            avg_temp|            avg_airp|           avg_humid|          avg_precip|
+----+---+---+------------+--------+-------------+------------+--------------------+--------------------+--------------------+--------------------+
|1001| 42|-73|      Agawam|      MA|Massachusetts|     Hampden|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.125000000000000000|
|1002| 42|-72|     Amherst|      MA|Massachusetts|   Hampshire|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.125000000000000000|
|1005| 42|-72|       Barre|      MA|Massachusetts|   Worcester|270.7250000000000...|1014.960000000000...|71.75250000000000...|0.135000000000000000|
|1029| 42|-73|   East Otis|      MA|Massachusetts|   Berkshire|270.8050000000000...|1016.122500000000...|72.2150

In [ ]:
county_df = county_weath.select('*').collect()
county = pd.DataFrame(county_df, columns=['zip','lat','lng','city','state_id','state_name','county_name','avg_temp','avg_airp','avg_humid','avg_precip'])
counties = county.dropna()
counties.head(2)

,zip,lat,lng,city,state_id,state_name,county_name,avg_temp,avg_airp,avg_humid,avg_precip
0,1001,42,-73,Agawam,MA,Massachusetts,Hampden,270.805000000000000000,1016.122500000000000000,72.215000000000000000,0.125000000000000000
1,1002,42,-72,Amherst,MA,Massachusetts,Hampshire,270.805000000000000000,1016.122500000000000000,72.215000000000000000,0.125000000000000000


In [ ]:
counties.dtypes

zip             int64
lat             int64
lng             int64
city           object
state_id       object
state_name     object
county_name    object
avg_temp       object
avg_airp       object
avg_humid      object
avg_precip     object
dtype: object

In [ ]:
mer_df = pd.DataFrame(allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','success').filter(allwines.variety == 'Merlot').collect(), columns=['lat','lng','avg_temp','avg_airp','avg_humid','avg_precid','success'], dtype=float)
rose_df = pd.DataFrame(allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','success').filter(allwines.variety == 'Rosé').collect(), columns=['lat','lng','avg_temp','avg_airp','avg_humid','avg_precid','success'], dtype=float)
syr_df = pd.DataFrame(allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','success').filter(allwines.variety == 'Syrah').collect(), columns=['lat','lng','avg_temp','avg_airp','avg_humid','avg_precid','success'], dtype=float)
savyb_df = pd.DataFrame(allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','success').filter(allwines.variety == 'Sauvignon Blanc').collect(), columns=['lat','lng','avg_temp','avg_airp','avg_humid','avg_precid','success'], dtype=float)
ries_df = pd.DataFrame(allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','success').filter(allwines.variety == 'Riesling').collect(), columns=['lat','lng','avg_temp','avg_airp','avg_humid','avg_precid','success'], dtype=float)
cabsav_df = pd.DataFrame(allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','success').filter(allwines.variety == 'Cabernet Sauvignon').collect(), columns=['lat','lng','avg_temp','avg_airp','avg_humid','avg_precid','success'], dtype=float)
char_df = pd.DataFrame(allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','success').filter(allwines.variety == 'Chardonnay').collect(), columns=['lat','lng','avg_temp','avg_airp','avg_humid','avg_precid','success'], dtype=float)
pino_df = pd.DataFrame(allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','success').filter(allwines.variety == 'Pinot Noir').collect(), columns=['lat','lng','avg_temp','avg_airp','avg_humid','avg_precid','success'], dtype=float)

In [ ]:
mer_df.dtypes

lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precid    float64
success       float64
dtype: object

In [ ]:
#features and targets
variety = mer_df.copy()
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7768817204301075

In [ ]:
#using model to predict the counties
Xcount = counties.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
mer_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = rose_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.8263403263403264

In [ ]:
#using model to predict the counties
Xcount = counties.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
rose_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = syr_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.6832993890020367

In [ ]:
#using model to predict the counties
Xcount = counties.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
syr_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = savyb_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7627118644067796

In [ ]:
#using model to predict the counties
Xcount = counties.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
savyb_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = ries_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7282693813625685

In [ ]:
#using model to predict the counties
Xcount = counties.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
ries_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = cabsav_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7054017249205629

In [ ]:
#using model to predict the counties
Xcount = counties.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
cabsav_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = char_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.6952212389380531

In [ ]:
#using model to predict the counties
Xcount = counties.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
char_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = pino_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.6450490351154697

In [ ]:
#using model to predict the counties
Xcount = counties.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
pino_preds = rf.predict(Xcount)

In [ ]:
counties_preds = counties.copy()
counties_preds['mer_success'] = mer_preds
counties_preds['rose_success'] = rose_preds
counties_preds['syr_success'] = syr_preds
counties_preds['savyb_success'] = savyb_preds
counties_preds['ries_success'] = ries_preds
counties_preds['cabsav_success'] = cabsav_preds
counties_preds['char_success'] = char_preds
counties_preds['pinot_success'] = pino_preds

In [ ]:
counties_preds.head()

,zip,lat,lng,city,state_id,state_name,county_name,avg_temp,avg_airp,avg_humid,avg_precip,mer_success,rose_success,syr_success,savyb_success,ries_success,cabsav_success,char_success,pinot_success
0,1001,42,-73,Agawam,MA,Massachusetts,Hampden,270.805000000000000000,1016.122500000000000000,72.215000000000000000,0.125000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1002,42,-72,Amherst,MA,Massachusetts,Hampshire,270.805000000000000000,1016.122500000000000000,72.215000000000000000,0.125000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1005,42,-72,Barre,MA,Massachusetts,Worcester,270.725000000000000000,1014.960000000000000000,71.752500000000000000,0.135000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1029,42,-73,East Otis,MA,Massachusetts,Berkshire,270.805000000000000000,1016.122500000000000000,72.215000000000000000,0.125000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1054,42,-72,Leverett,MA,Massachusetts,Franklin,270.805000000000000000,1016.122500000000000000,72.215000000000000000,0.125000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
#counties_preds.to_csv('county_predictions.csv')

In [ ]:
spark_counties = spark.createDataFrame(counties_preds)

In [ ]:
spark_counties.write.jdbc(url=jdbc_url, table='county_predictions', mode=mode, properties=config)

In [ ]:
sparked_preds = spark.read.jdbc(url=jdbc_url, table='county_predictions', properties=config)

In [ ]:
sparked_preds.show()

+-----+---+---+--------------+--------+----------+-------------+--------------------+--------------------+--------------------+--------------------+-----------+------------+-----------+-------------+------------+--------------+------------+-------------+
|  zip|lat|lng|          city|state_id|state_name|  county_name|            avg_temp|            avg_airp|           avg_humid|          avg_precip|mer_success|rose_success|syr_success|savyb_success|ries_success|cabsav_success|char_success|pinot_success|
+-----+---+---+--------------+--------+----------+-------------+--------------------+--------------------+--------------------+--------------------+-----------+------------+-----------+-------------+------------+--------------+------------+-------------+
|50515| 43|-95|      Ayrshire|      IA|      Iowa|    Palo Alto|263.4875000000000...|1013.162500000000...|80.39500000000000...|0.015000000000000000|        0.0|         0.0|        0.0|          0.0|         0.0|           0.0|        

In [ ]:
sparked_preds.where(sparked_preds.mer._success == 1).count()

48